In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import keras
import numpy as np 
import pandas as pd 
import zipfile
import matplotlib.pyplot as plt
import os
import shutil

In [ ]:
shutil.unpack_archive('../input/dogs-vs-cats/train.zip','./')
shutil.unpack_archive('../input/dogs-vs-cats/test1.zip','./')

Another way of unzipping files is :

In [ ]:
#with zipfile.ZipFile("../input/dogs-vs-cats/train.zip", 'r') as zip_ref:
                #zip_ref.extractall("./")

#with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip", 'r') as zip_ref:
                #zip_ref.extractall("./")

In [ ]:
print("Total train images")
print(len([files for files in os.listdir('./train') if os.path.isfile(os.path.join('./train', files))]))
print("Total test images")
print(len([files for files in os.listdir('./test1') if os.path.isfile(os.path.join('./test1', files))]))

In [ ]:
print("Total no of cat images in train dataset:")
print(len([files for files in os.listdir('./train') if os.path.isfile(os.path.join('./train', files)) and 'cat' in files]))
print("Total no of dog images in train dataset:")
print(len([files for files in os.listdir('./train') if os.path.isfile(os.path.join('./train', files)) and 'dog' in files]))

In [ ]:
print("Total no of cat images in test dataset:")
print(len([files for files in os.listdir('./test1') if os.path.isfile(os.path.join('./test1', files)) and 'cat' in files]))
print("Total no of dog images in test dataset:")
print(len([files for files in os.listdir('./test1') if os.path.isfile(os.path.join('./test1', files)) and 'dog' in files]))

Since images in test dataset are unlabelled, hence model can't find any 'cat' and 'dog' images

**Showing a random image in train dataset**

In [ ]:
for img in os.listdir('./train'):
    img_path = os.path.join('./train',img)
    print(img_path)
    img_arr = plt.imread(img_path)
    plt.imshow(img_arr)
    break

**Showing a random image from test dataset**

In [ ]:
for img in os.listdir('./test1'):
    img_path = os.path.join('./test1',img)
    print(img_path)
    img_arr = plt.imread(img_path)
    plt.imshow(img_arr)
    break

Lets now prepare the data !


## Image Data Pre-processing

**Lets create 3 directories each for training, testing and validation**

In [ ]:
 base_dir = './'

train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)
val_dir = os.path.join(base_dir, 'val_dir')
os.mkdir(val_dir)
test_dir = os.path.join(base_dir, 'test_dir')
os.mkdir(test_dir)

train_cats = os.path.join(train_dir, 'cats')
os.mkdir(train_cats) 
train_dogs = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs)

val_cats = os.path.join(val_dir, 'cats')
os.mkdir(val_cats) 
val_dogs = os.path.join(val_dir, 'dogs')
os.mkdir(val_dogs)

test_cats = os.path.join(test_dir, 'cats')
os.mkdir(test_cats) 
test_dogs = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs)

In [ ]:
files = ['cat.{}.jpg'.format(i) for i in range(8750)]
for filename in files:
    src = os.path.join('./train', filename)
    dst = os.path.join(train_cats, filename)
    shutil.copyfile(src, dst)
    
files = ['cat.{}.jpg'.format(i) for i in range(8750, 12500)]
for filename in files:
    src = os.path.join('./train', filename)
    dst = os.path.join(val_cats, filename)
    shutil.copyfile(src, dst)
    

    
files = ['dog.{}.jpg'.format(i) for i in range(8750)]
for filename in files:
    src = os.path.join('./train', filename)
    dst = os.path.join(train_dogs, filename)
    shutil.copyfile(src, dst)

files = ['dog.{}.jpg'.format(i) for i in range(8750, 12500)]
for filename in files:
    src = os.path.join('./train', filename)
    dst = os.path.join(val_dogs, filename)
    shutil.copyfile(src, dst)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
    zoom_range = 0.2,
    horizontal_flip = True,
    rotation_range = 30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2,
    rescale=1./255)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
IMAGE_SIZE = (120,120)
BATCH_SIZE = 128


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    class_mode='binary')   



val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = 'binary')

In [ ]:
for data_batch, label_batch in train_generator:
    print("Train data batch shape:", data_batch.shape)
    print("Train label batch shape:", label_batch.shape)
    break;
    
for data_batch, label_batch in val_generator:
    print("Val data batch shape:", data_batch.shape)
    print("Val label batch shape:", label_batch.shape)
    break;

In [ ]:
for data_batch, label_batch in train_generator:
    print("Train data batch shape:", data_batch.shape)
    print("Train label batch shape:", label_batch.shape)
    break;

> Data batch shape : No.of batches x Img_size x no.of channels

# VGG-16 MODEL

In [ ]:
from keras.applications.vgg16 import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(120,120, 3))

In [ ]:
conv_base.summary()

In [ ]:
conv_base.trainable = False

In [ ]:
from keras.models import Sequential
from keras import layers

model = Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

Always use 'sigmoid' in the final layer for binary classification. If we use 'softmax', the loss will become very big and hard to converge.

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
             optimizer = keras.optimizers.Adam(),
             metrics = ['acc'])

Adding callbacks

In [ ]:
callbacks_list = [keras.callbacks.EarlyStopping(
        monitor = 'acc',
        patience = 2),
                  keras.callbacks.ModelCheckpoint(
        filepath='vgg16_standart.h5',
        monitor='loss',
        save_best_only=True)]

In [ ]:
train_generator.n

In [ ]:
model_training = model.fit(
    train_generator,
    validation_data=val_generator,
    steps_per_epoch=train_generator.n//BATCH_SIZE,
    validation_steps=val_generator.n//BATCH_SIZE,
    epochs = 5,
    callbacks = callbacks_list)

Our model name is 'model'.
The History object gets returned by the fit method of models. Hence I assigned it as different name 'model_training' where the History object gets stored.

Go to this link to know more about keras.callbacks.History   http://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History

In [ ]:
acc = model_training.history['acc']
val_acc=model_training.history['val_acc']
loss = model_training.history['loss']
val_loss = model_training.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'o', label='Training accuracy')
plt.plot(epochs, val_acc, 'bo', label='Validation accuracy')

plt.title('Training accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'o', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='Validation loss')
plt.title('Training loss')
plt.legend()

# DenseNet201

In [ ]:
from keras.applications import DenseNet201

conv_base = DenseNet201(weights='imagenet',
                 include_top=False,
                 input_shape=(120,120, 3))

In [ ]:
conv_base.summary()

In [ ]:
conv_base.trainable = False

In [ ]:
from keras.models import Sequential
from keras import layers

model_DN = Sequential()
model_DN.add(conv_base)
model_DN.add(layers.Flatten())
model_DN.add(layers.Dense(256, activation='relu'))
model_DN.add(layers.Dense(1, activation = 'sigmoid'))

In [ ]:
model_DN.summary()

In [ ]:
model_DN.compile(loss='binary_crossentropy',
             optimizer = keras.optimizers.Adam(),
             metrics = ['acc'])

In [ ]:
callbacks_list = [keras.callbacks.EarlyStopping(
        monitor = 'acc',
        patience = 1),
                  keras.callbacks.ModelCheckpoint(
        filepath='densenet201.h5',
        monitor='loss',
        save_best_only=True)]

In [ ]:
train_generator.n

In [ ]:
model_training = model_DN.fit(
    train_generator,
    validation_data=val_generator,
    steps_per_epoch=train_generator.n//BATCH_SIZE,
    validation_steps=val_generator.n//BATCH_SIZE,
    epochs = 5,
    callbacks = callbacks_list)

In [ ]:
acc = model_training.history['acc']
val_acc= model_training.history['val_acc']
loss =  model_training.history['loss']
val_loss =  model_training.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'o', label='Training accuracy')
plt.plot(epochs, val_acc, 'bo', label='Validation accuracy')

plt.title('Training accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'o', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='Validation loss')
plt.title('Training loss')
plt.legend()

# Test Data